In [1]:
!pip install langchain-community langchain-core  
# 0.1.48
!pip install langchain_openai===0.1.5
!pip install langsmith==0.1.53
%pip install --upgrade --quiet  azure-search-documents
%pip install --upgrade --quiet  azure-identity
! pip install azure-search-documents --pre 
! pip install openai==1.3.3
! pip install python-dotenv
! pip install langchain===0.1.17
# !pip install langchain --upgrade 
!pip install langchain-community===0.0.36

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 1.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: openai
    Found existing installation: openai 1.3.3
    Uninstalling openai-1.3.3:
      Successfully uninstalled openai-1.3.3
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached openai-1.3.3-py3-none-any.whl (220 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.30.4
    Uninstalling openai-1.30.4:
      Successfully uninstalled openai-1.30.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.1.5 requires openai<2.0.0,>=1.10.0, but you have openai 1.3.3 which is incompatible.


In [2]:
import os

from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_openai import AzureOpenAIEmbeddings

In [3]:
!pip list

Package                  Version
------------------------ ------------
aiohttp                  3.8.4
aiosignal                1.3.1
altair                   5.0.1
anyio                    3.5.0
appdirs                  1.4.4
appnope                  0.1.2
argon2-cffi              21.3.0
argon2-cffi-bindings     21.2.0
asttokens                2.0.5
async-timeout            4.0.2
attrs                    22.1.0
azure-common             1.1.28
azure-core               1.29.6
azure-cosmos             4.5.1
azure-identity           1.16.0
azure-search             1.0.0b2
azure-search-documents   11.4.0
Babel                    2.11.0
backcall                 0.2.0
backoff                  2.2.1
beautifulsoup4           4.12.2
bleach                   4.1.0
blinker                  1.6.2
bokeh                    3.4.1
Bottleneck               1.3.5
brotlipy                 0.7.0
cachetools               5.3.1
certifi                  2023.5.7
cffi                     1.15.1
charset-normali

# Azure Cognitive Search LangChain Vector Code Sample
This code demonstrates how to use Azure Cognitive Search with OpenAI and the Azure Cognitive Search LangChain Vector Store
To run the code, install the following packages. Please use the latest pre-release version `pip install azure-search-documents==11.4.0b6` until the Langchain AzureSearch vector store integration gets updated with the latest python azure sdk updates`.

In [3]:
from dotenv import dotenv_values
from azure.identity import DefaultAzureCredential
import os

env_name = "cosmos.env"
config = dotenv_values(env_name)

cosmosdb_endpoint = config['cosmos_db_api_endpoint']
cosmosdb_key = config['cosmos_db_api_key']
cosmosdb_connection_str = config['cosmos_db_connection_string']

endpoint = config['cognitive_search_api_endpoint']
key_credential = config['cognitive_search_api_key'] if len(config['cognitive_search_api_key']) > 0 else None
index_name = "cosmos-vector-index"
azure_openai_endpoint = config['openai_api_endpoint']
azure_openai_key = config['openai_api_key'] if len(config['openai_api_key']) > 0 else None
azure_openai_embedding_deployment = config['openai_embeddings_deployment']
embedding_model_name = config['openai_completions_deployment']
azure_openai_api_version = config['openai_api_version']

completions_deployment = config['openai_completions_deployment']

AZURE_OPENAI_API_KEY=azure_openai_key

In [4]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    SearchIndex
)
from azure.core.credentials import AzureKeyCredential

credential = AzureKeyCredential(key_credential)

## Import required libraries and environment variables

In [5]:
# Import required libraries  
import openai
import os  
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings

from langchain.chat_models import AzureChatOpenAI

from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate


In [6]:
from azure.search.documents.indexes.models import (
    SemanticSearch,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField
)

 ## Configure Vector Store Settings

In [7]:
# vector_store_address: str = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")  
# vector_store_password: str = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
vector_store_address: str = config['cognitive_search_api_endpoint']  
vector_store_password: str = config['cognitive_search_api_key']

## Configure OpenAI Settings

In [8]:
llm = AzureChatOpenAI(deployment_name='conv_ai-dm', 
                      model_name='gpt-35-turbo',
                      api_key=azure_openai_key,
                      api_version='2024-02-15-preview',
                      azure_endpoint='https://openai-ccat-poc1.openai.azure.com/',)

/Users/agupt38/anaconda3/envs/env_v2_3_11/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


## Create embeddings and vector store instances
Read your data, generate OpenAI embeddings and export to a format to insert your Azure Cognitive Search index:

In [9]:
from langchain.vectorstores.azuresearch import AzureSearch

In [ ]:
# embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
#     azure_deployment=azure_openai_embedding_deployment,
#     openai_api_version=azure_openai_api_version,
#     azure_endpoint=azure_openai_endpoint,
#     api_key=azure_openai_key,
# )

In [13]:
embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    azure_deployment=completions_deployment,
    openai_api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key,
)

In [14]:
index_name: str = "document-index-hausman"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=endpoint,
    azure_search_key=key_credential,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

In [12]:
index_name: str = "document-index-lakeland"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=endpoint,
    azure_search_key=key_credential,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

NotFoundError: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}

## Insert text and embeddings into vector store

In [15]:
!pip install pypdf

In [17]:
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

#loader = TextLoader("../data/state_of_the_union.txt", encoding="utf-8")
# loader = PyPDFLoader("Lakeland Regional Med Center_FL_Geren_E_UHC_CIRAM0011705894_OI67475651_PATNTRECRD_DRG.PDF")
loader = PyPDFLoader("./MR_ops.PDF.ocr 1.pdf")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
docs = text_splitter.split_documents(documents)

vector_store.add_documents(documents=docs)

['NmQ4NWMzY2UtMDcxNS00OGUzLWI1MWQtYjE4YjA0OTc2ZDQ2',
 'NTE4MDlhOTMtODAyYy00Yzg1LTk1OWYtMzI4YzA3ZDgzNGIw',
 'YzdiM2QxNzYtN2Y5NC00ZWRiLTk0YmEtMDY2NGJmODJjYzk2',
 'Y2VhMmYzMmUtYjE3Mi00ZjdiLWJiYmEtNmQ5ODMwN2FmMDA4',
 'MGVmODBlN2UtZDRkZS00NWEzLWE2MDYtZjA3MTc4NmViYWZk',
 'OGE1YWJkZDAtYTYyZS00MWU3LTg4NzQtMDIyMjRiYWM1Y2Ez',
 'ZWI0OWY3NmUtMjVmYy00MTk0LWI5OTEtZmQ3Zjg0MjgyZmZm',
 'YmZiNTNkMzYtNGY2NC00YTg0LWFjYjktYWE5NzUyNWVmMDY0',
 'YTRlNDhlOTktMWFhNS00OGRjLWI4ODMtYTZlZTkwNzc0M2Zi',
 'YzMxYjVlNDktY2JjOC00ODI4LWFkYjgtZWZmZjI2OTVjODBk']

In [18]:
from langchain_community.retrievers import AzureAISearchRetriever
retriever = AzureAISearchRetriever(
    content_key="content", top_k=1, index_name="document-index-hausman",
    service_name = 'case-cog-search',
    api_key = key_credential
)

## Querying the data 

In [78]:
# retriever.invoke("what is history for patient?")

[Document(page_content="Patient Name: Hausman, Hensley M (MR#009419780) Date of Birth: 6/23/2021 12/13/23, 1:38 AM \nHausman, Hensley M (MR # 009419780) DOB: 06/23/2021 \nSocial History - Place of Residence \n· Living arrangement \n· Location \n, \nReview of Systems \nReview of Systems \nConstitutional: Negative for chills and fever. \nHENT: Negative for ear pain and sore throat. \nEyes: Negative for pain and redness. \nRespiratory: Negative for cough and wheezing. \nCardiovascular: Negative for chest pain and leg swelling. \nGastrointestinal: Negative for abdominal pain and vomiting. \nGenitourinary: Negative for frequency and hematuria. \nMusculoskeletal: Negative for gait problem and joint swelling. \nSkin: Positive for wound. Negative for color change and rash. \nNeurological: Negative for seizures and syncope. \nAll other systems reviewed and are negative. \nPhysical Exam \nED Triage Vitals \nTemp \n10/29/23 1500 \n36.4 °■ (97.6 °F) 120 Pulse \n10/29/23 1500 Resp \n10/29/23 1500 \

In [19]:


CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, 
rephrase the follow up question to be a standalone question.
Follow Up Input: {question}
Standalone question:""")

qa = ConversationalRetrievalChain.from_llm(llm = llm,
                                           retriever=retriever,
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True,
                                           verbose=False)

In [23]:
chat_history = []
# query = "Give me a summary for the patient medical record"
query = "What is physical exam of patient"
result = qa({"question": query, "chat_history": chat_history})

print("Question:", query)
print("Answer:", result["answer"])

Question: What is physical exam of patient
Answer: The physical exam of the patient includes the following findings:

Constitutional:
- General: The patient is active and not in acute distress. She is a well-developed toddler white female in her mom's arms. She is awake and alert.

HENT:
- Head: Normocephalic.
- Lips: She has a 0.75 cm superficial laceration on the left upper lip, crossing the vermilion border in a triangle-shaped flap-like pattern. There is no active bleeding, swelling, or intraoral laceration. The dentition is intact.

Eyes:
- No specific information is provided about the eyes in the given context.

Mouth/Throat:
- Mouth: Mucous membranes are moist.
- Pharynx: Oropharynx is clear with no oropharyngeal exudate.
